In [ ]:
%load_ext autoreload

In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import numpy as np
import matplotlib.pyplot as plt
import ceedub as cw

from scipy.signal import argrelmax
from scipy.optimize import minimize_scalar

%matplotlib inline
%autoreload 2

# Definitions

In [ ]:
GMsun = 1.32712440018e20  # m^3/s^2
c = 299792458 # m/s

Rsun = GMsun / c**2
Tsun = GMsun / c**3

## Newtonian orbital parameters

In [ ]:
def dvp_N(vp, de, eta):
    """Newtonian change in pericenter speed
    eq. 71 of L&Y
    """
    return -13*np.pi/96 * eta * vp**6 * (1 + 44/65*de)

def dde_N(vp, de, eta):
    """Newtonian change in eccentricity
    eq. 72 of L&Y
    """
    return 85*np.pi/48 * eta * vp**5 * (1 + 791/850*de)

def T_N(vp, de, Mtot=1):
    """Newtonian orbital period (time between bursts)
    eq. 76 of L&Y
    """
    return 2*np.pi * Mtot / vp**3 * ((2-de)/de)**(3/2)

def f_N(vp, de, Mtot=1):
    """Newtonian GW frequency of burst
    eq. 79 of L&Y
    """
    return vp**3 / (2*np.pi*Mtot * (2-de))

## second order ($\frac{v^2}{c^2}$) corrections for orbits

In [ ]:
def V2(de, eta):
    """1PN correction to change in pericenter speed
    order (v/c)^2
    eq. 73 of L&Y
    """
    return -251/104*eta + 8321/2080 + de*(14541/6760*eta - 98519/135200)

def D2(de, eta):
    """1PN correction to change in eccentricity
    order (v/c)^2
    eq. 74 of L&Y
    """
    return -4017/680*eta + 4773/800 + de*(225393/144500*eta - 602109/340000)

def P2(de, eta):
    """1PN correction to orbital period (time between bursts)
    order (v/c)^2
    eq. 77 of L&Y
    """
    return 3/2*eta - 3/4 + de*(-5/8*eta + 3/4)

def R2(de, eta):
    """1PN correction to GW frequency
    order (v/c)^2
    eq. 80 of L&Y
    """
    return 7/4*eta - 5/2 - 5/8*de*eta

## 1PN orbital parameters (next given prev)

In [ ]:
def dvp_1PN(vp0, de0, eta):
    """1PN change in pericenter speed
    eq. 69 of L&Y
    """
    return dvp_N(vp0, de0, eta) * (1 + V2(de0, eta)*vp0**2)

def dde_1PN(vp0, de0, eta):
    """1PN change in eccentricity
    eq. 70 of L&Y
    """
    return dde_N(vp0, de0, eta) * (1 + D2(de0, eta)*vp0**2)

def T_PN(vp0, de0, eta, Mtot=1):
    """1PN orbital period (time between bursts)
    eq. 75 of L&Y
    """
    vp1 = vp0 + dvp_1PN(vp0, de0, eta)
    de1 = de0 + dde_1PN(vp0, de0, eta)
    return T_N(vp1, de1, Mtot) * (1 + P2(de1, eta)*vp1**2)

def f_PN(vp0, de0, eta, Mtot=1):
    """1PN GW frequency of burst
    eq. 78 of L&Y
    """
    vp1 = vp0 + dvp_1PN(vp0, de0, eta)
    de1 = de0 + dde_1PN(vp0, de0, eta)
    return f_N(vp1, de1, Mtot) * (1 - R2(de1, eta)*vp1**2)

# testing

In [ ]:
tf_correct = [[-1754.0, 0.021160242920888948],
              [-614.0, 0.023075408530304677],
              [-203.0, 0.027441440005832807],
              [0.0, 0.05255602595335716]
             ]

In [ ]:
Mtot = 30
q = 0.25

eta = q / (1+q)**2

In [ ]:
t_star, f_star = tf_correct[2]
t_next, f_next = tf_correct[3]

In [ ]:
def diff_for(de):
    vp = (2*np.pi*f_star * (2-de))**(1/3)
    t = t_star + T_PN(vp, de, eta)
    f = f_PN(vp, de, eta)
    return np.abs(t-t_next)

In [ ]:
des = np.arange(0.01, 0.99, 1e-2)
diff = [diff_for(x) for x in des]
plt.plot(des, diff)

In [ ]:
de + dde_1PN(vp, de, eta)

In [ ]:
de = 0.95
vp = (2*np.pi*f_star * (2-de))**(1/3)
t = t_star + T_PN(vp, de, eta)
f = f_PN(vp, de, eta)

In [ ]:
t,f

In [ ]:
f_next

In [ ]:
vps = np.arange(0.05, 0.95, 0.01)
des = np.arange(0.01, 0.50, 0.01)
period = np.zeros([len(vps), len(des)])

for ii,vp in enumerate(vps):
    for jj,de in enumerate(des):
        period[ii,jj] = T_N(vp, de)

In [ ]:
np.log10(np.diff(np.array(tf_correct).T[0]))

In [ ]:
dT = np.log10(np.diff(np.array(tf_correct).T[0]))

#plt.pcolormesh(des, vps, np.log10(period))
plt.contourf(des, vps, np.log10(period))#, colors='k')
plt.colorbar()
plt.scatter([0.25, 0.3, 0.35, 0.4], [0.83, 0.75, 0.68, 0.63], marker='x', s=100, color='r')

plt.ylabel(r"$v/c$")
plt.xlabel(r"$\delta e = 1-e$")

In [ ]:
np.diff(np.array(tf_correct).T[0])

In [ ]:
de = 0.4
vp = (2*np.pi*f_star * (2-de))**(1/3)
print("de = {:.2f}, vp = {:.3f}".format(de, vp))

vp=0.53
print(T_N(vp, de))
print(T_PN(vp, de, eta))

In [ ]:
from scipy.optimize import minimize

Mtot = 30
q = 0.25

eta = q / (1+q)**2

t_star, f_star = tf_correct[2]
t_next, f_next = tf_correct[3]

sigT = 5  # hit 203 +/- 5
sigF = 0.005  # hit 0.052 +/- 0.005

def diff_for(x):
    vp, de = x
    t = t_star + T_PN(vp, de, eta)
    f = f_PN(vp, de, eta)
    return (t-t_next)**2/sigT**2 + (f-f_next)**2/sigF**2

guess = [0.50, 0.40]
cnstrnt = [{'type':'ineq', 'fun':lambda x: x[0]},
           {'type':'ineq', 'fun':lambda x: x[1]}]  # both params must be non-negative
result = minimize(diff_for, guess, method='SLSQP', constraints=cnstrnt)
print(result.x)

In [ ]:
vp, de = result.x

In [ ]:
T_PN(vp, de, eta)

In [ ]:
f_PN(vp, de, eta)

In [ ]:
tf_correct

In [ ]:
fun = lambda x: x[1]
fun(result.x)